# running socioeconomic results with cdc subset

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px # choropleth maps
from IPython.display import Image
import os
import sys
import json
# plotly maps
# import plotly.figure_factory as ff
# from urllib.request import urlopen

# sys.path.append(os.path.abspath("C:/Users/Alex/Documents/My Data Files/COVID Research/analysis/mention_2_hashtag.py"))

# from analysis.functions.mention_2_hashtag import * 
# from analysis.functions.datamanip import *

# n grams
from sklearn.feature_extraction.text import CountVectorizer

raw = pd.read_csv('flair.joined.tweets.csv')

# changing date to more readable format
raw['created_at'] = pd.to_datetime(raw['created_at'])

# getting seperated date and time columns
raw['date'] = raw['created_at'].dt.date
raw['week'] = raw['created_at'].dt.week
raw['time'] = raw['created_at'].dt.time
raw['hour'] = raw['created_at'].dt.hour

min_wk = raw['week'].min()
max_wk = raw['week'].max()

min_date = raw['date'].min()
max_date = raw['date'].max()

# changing na to None
raw = raw.fillna('None')

raw = raw[raw['sentiment'] != 'None']

raw['is_negative'] = [1 if 'NEGATIVE' in str(sent) else 0 for sent in raw['sentiment']]
raw['is_positive'] = [1 if 'POSITIVE' in str(sent) else 0 for sent in raw['sentiment']]


def convert_fips(unknown_fips_list, fin):
    
    def convert(fipsline):
        name, postal, fips= fipsline.strip().split('\t')

        return {fips: name}

    fips_dict = {f: n 
                for dic in [convert(line)for line in open(fin, 'r')]
                for f, n in dic.items()}

    return [fips_dict[f'{unknown:02d}'] for unknown in unknown_fips_list]

raw = raw[raw['statefips'] != 'None']

raw['state_name'] = convert_fips([int(float(fips)) for fips in raw['statefips']],
                     '../analysis/functions/state.fips.txt')

us_state_abbrev = json.load(open('state_abbreviations.json'))

raw['state_abv'] = [us_state_abbrev[s] for s in raw['state_name']]

raw['fips'] = [f'{int(float(state)):02d}{int(float(county)):03d}' for i, state, county in raw[['statefips', 'countyfips']].itertuples()]
raw['count'] = 1
# raw.head()

all_splits = pd.to_datetime(['2020-03-05', '2020-03-26', '2020-05-01'])
raw['stage'] = ['s:1' if date < all_splits[0] else 's:2' if date < all_splits[1] else 's:3' if date < all_splits[2] else 's:4' for date in raw['date']]



#########################
# thresholds
thresholds = [0.8, 0.9, 0.95]
for thresh in thresholds:
    raw[f't:{thresh}'] = [1 if float(conf) > thresh else 0 for conf in raw['sent_confidence']]

col = 't:0.8'
raw['category'] = ['NEUT' if t == 0 else 'POS' if p == 1 else 'NEG' for i, n, p, t in raw[['is_negative', 'is_positive', col]].itertuples()]
raw['is_neutral'] = [1 if cat == 'NEUT' else 0 for cat in raw['category']]
raw['is_positive'] = [1 if cat == 'POS' else 0 for cat in raw['category']]
raw['is_negative'] = [1 if cat == 'NEG' else 0 for cat in raw['category']]

raw['is_neutral2'] = raw['is_neutral']
raw['is_positive2'] = raw['is_positive']
raw['is_negative2'] = raw['is_negative']

raw['is_neutral3'] = raw['is_neutral']
raw['is_positive3'] = raw['is_positive']
raw['is_negative3'] = raw['is_negative']

##################
# cdc

keywords = ['social distancing', 'social distance', 'physical distance', '6 feet', 'stay at home', 'school isolation', 'isolation', 'stay home', 'avoid touching', 'mask', 'covering', 'face shield', 'wear a mask', 'surgical mask', 'N95 respirator', 'wearing gloves', 'face shields', 'facial covering', 'skin protection', 'eye protection', 'ppe', 'wash hands', 'hand sanitizer', 'disinfect', 'clean', 'detergent', 'handwashing', 'hand hygiene', 'prevention hygiene', 'sprays', 'concentrates', 'wipes', 'routine cleaning', 'bleach solution', 'test', 'business closure']

topics = []
indexes = {i:0 for i in range(len(raw.index))}

for i, topic in enumerate(keywords):
    topics.append([1 if topic in text else 0 for text in raw['ogtext']])

for i, kw in enumerate(topics):
    for data_ind, value in enumerate(topics[i]):
        indexes[data_ind] += value

raw['in_cdc'] = [indexes[i] if 0 <= indexes[i] <= 1 else 1 for i in indexes]

cdc = raw[raw['in_cdc'] == 1]

cdc_splits = pd.to_datetime(['2020-03-12', '2020-04-20', '2020-05-14'])
cdc['stage'] = ['s:1' if date < cdc_splits[0] else 's:2' if date < cdc_splits[1] else 's:3' if date < cdc_splits[2] else 's:4' for date in cdc['date']]

# print(f'there are a total of {len(cdc.index)} tweets after subsetting for cdc keywords')
# print(raw)
# print(cdc)
print('loaded...')

loaded...


# subsetting for cdc keywords

In [2]:
keywords = ['social distancing', 'social distance', 'physical distance', '6 feet', 'stay at home', 'school isolation', 'isolation', 'stay home', 'avoid touching', 'mask', 'covering', 'face shield', 'wear a mask', 'surgical mask', 'N95 respirator', 'wearing gloves', 'face shields', 'facial covering', 'skin protection', 'eye protection', 'ppe', 'wash hands', 'hand sanitizer', 'disinfect', 'clean', 'detergent', 'handwashing', 'hand hygiene', 'prevention hygiene', 'sprays', 'concentrates', 'wipes', 'routine cleaning', 'bleach solution', 'test', 'business closure']

topics = []
indexes = {i:0 for i in range(len(raw.index))}

for i, topic in enumerate(keywords):
    topics.append([1 if topic in text else 0 for text in raw['ogtext']])

for i, kw in enumerate(topics):
    for data_ind, value in enumerate(topics[i]):
        indexes[data_ind] += value

raw['in_cdc'] = [indexes[i] if 0 <= indexes[i] <= 1 else 1 for i in indexes]

cdc = raw[raw['in_cdc'] == 1]

print(f'there are a total of {len(cdc.index)} tweets after subsetting for cdc keywords')

there are a total of 53272 tweets after subsetting for cdc keywords


# income df load

In [3]:
income = pd.read_csv('fips_unemp_medhh.csv')
income.head()

,FIPStxt,Stabr,area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2018,Med_HH_Income_Percent_of_State_Total_2018
0,0,US,United States,NaN,NaN,NaN,"142,601,667","136,904,680","5,696,987",4.0,...,"161,389,026","155,102,319","6,286,707",3.9,"163,100,055","157,115,247","5,984,808",3.7,"61,937",NaN
1,1000,AL,Alabama,NaN,NaN,NaN,"2,133,223","2,035,594","97,629",4.6,...,"2,216,627","2,130,845","85,782",3.9,"2,241,747","2,174,483","67,264",3.0,"49,881",100.0
2,1001,AL,"Autauga County, AL",2.0,2.0,1.0,"21,720","20,846",874,4.0,...,"26,196","25,261",935,3.6,"26,172","25,458",714,2.7,"59,338",119.0
3,1003,AL,"Baldwin County, AL",3.0,2.0,1.0,"69,533","66,971","2,562",3.7,...,"95,233","91,809","3,424",3.6,"97,328","94,675","2,653",2.7,"57,588",115.5
4,1005,AL,"Barbour County, AL",6.0,6.0,0.0,"11,373","10,748",625,5.5,...,"8,414","7,987",427,5.1,"8,537","8,213",324,3.8,"34,382",68.9


# merge dataframes

In [4]:
stats_df = income[income['FIPStxt'].isin(raw['fips'].unique())][['FIPStxt', 'Unemployment_rate_2019', 'Median_Household_Income_2018', 'Med_HH_Income_Percent_of_State_Total_2018']]

stats_df.columns = ['fips', 'unemp_rate', 'median_hh', '%_state_total']

stats_df['median_hh'] = stats_df['median_hh'].replace(',','', regex = True)
stats_df['fips'] = stats_df['fips'].astype('str')
raw = pd.merge(cdc, stats_df, on='fips')

# had to pull peurto rico stats
raw = raw[~raw['median_hh'].isna()]
raw['median_hh'] = raw['median_hh'].astype('int')


raw.head()

,id_str,created_at,follower_count,friends_count,statefips,countyfips,countyname,is_urban,ogtext,cleantext,...,is_neutral2,is_positive2,is_negative2,is_neutral3,is_positive3,is_negative3,in_cdc,unemp_rate,median_hh,%_state_total
0,id:1226325348247310337,2020-02-09 02:02:01+00:00,1196.0,1661.0,39.0,117.0,Morrow,0,if they pulled the stuff in the sixties and se...,pulled stuff sixty seventy asse handed throw j...,...,0,0,1,0,0,1,1,4.1,60452,107.7
1,id:1233612267415097345,2020-02-29 04:37:38+00:00,823.0,1002.0,39.0,117.0,Morrow,0,so by correct handwashing amp other essential ...,correct handwash amp essential hygiene routine...,...,1,0,0,1,0,0,1,4.1,60452,107.7
2,id:1228783507033509890,2020-02-15 20:49:52+00:00,324.0,98.0,39.0,117.0,Morrow,0,this is a disaster waiting to happen,disaster waiting happen,...,0,0,1,0,0,1,1,4.1,60452,107.7
3,id:1227377430362492928,2020-02-11 23:42:37+00:00,3767.0,4007.0,39.0,117.0,Morrow,0,thousands of jobs went to because of bad polic...,thousand job go bad policy profit follow money...,...,0,0,1,0,0,1,1,4.1,60452,107.7
4,id:1233230888659451904,2020-02-28 03:22:10+00:00,156.0,162.0,39.0,117.0,Morrow,0,dear powers that be get these test kits develo...,dear power test kit develop mass produce send ...,...,1,0,0,1,0,0,1,4.1,60452,107.7


# stats on unemployment rate and median household income

In [19]:
print('median household income stats')
print(raw['median_hh'].describe())
print()
print('unemployment rate stats')
print(raw['unemp_rate'].describe())

median household income stats
count     40870.000000
mean      65723.964497
std       17698.484566
min       29380.000000
25%       52910.000000
50%       60862.000000
75%       76255.000000
max      140382.000000
Name: median_hh, dtype: float64

unemployment rate stats
count    40870.000000
mean         3.621304
std          0.883700
min          1.800000
25%          3.000000
50%          3.500000
75%          4.000000
max         12.200000
Name: unemp_rate, dtype: float64


# median household

In [18]:
counties = raw.groupby(['fips']).agg({'is_negative':'mean', 'is_positive':'mean', 'is_neutral':'mean', 'median_hh':'mean', 'unemp_rate':'mean', 'state_name':'max', 'count':'count'}).reset_index()

counties = counties[counties['count'] > 15]

# sns.regplot(x='median_hh', y='is_negative', data=counties, marker='+')

# unemployment rate

In [17]:
# sns.regplot(x='unemp_rate', y='is_negative', data=counties, marker='+')

# pearsons r

In [16]:
#  counties[['is_negative','unemp_rate','median_hh']].corr()

# pearsons r squared

In [15]:
#  counties[['is_negative','unemp_rate','median_hh']].corr()**2

# very low correlations compared to non-cdc subsetted data

In [14]:
import scipy

cats = 'negative positive neutral'.split(' ')

for cat in cats:
    x= counties[f'is_{cat}']
    y= counties['unemp_rate']

    print(f'{cat}:   r                  p')
    print(scipy.stats.pearsonr(x, y))    # Pearson's r
    print()

# print(scipy.stats.spearmanr(x, y))   # Spearman's rho

# print(scipy.stats.kendalltau(x, y))  # Kendall's tau


negative:   r                  p
(0.07396401802178429, 0.1692216453367042)

positive:   r                  p
(-0.1005989857272788, 0.06121476762571336)

neutral:   r                  p
(0.030663165019377214, 0.5691767706547305)

